In [1]:
import threading

import findspark
findspark.init()

import pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Partition Run").getOrCreate()

In [2]:
class Sentiment:
    Positive = "Positive"
    Negative = "Negative"
    Neutral = "Neutral"

class Review:
    def __init__(self,text,score):
        self.text = text
        self.score = score
        self.sentiment = self.getSentiment()
        
    def getSentiment(self):
        if (self.score <= 2):
            return Sentiment.Negative
        elif self.score == 3:
            return Sentiment.Neutral
        else:
            return Sentiment.Positive

In [3]:
data = spark.read.json("/home/shivam/intern/rev.json").select("overall","reviewText")
df = data.withColumn("overint",data["overall"].cast("Int"))
df = df.select("overint","reviewText")

In [4]:
d = df.rdd.map(lambda r : (r[0],r))

In [5]:
n = 5

def partitioner(key):
    if(key % n == 1):
        return 1
    elif(key % n == 2):
        return 2
    elif(key % n == 3):
        return 3
    elif(key % n == 4):
        return 4
    else:
        return 0
    
    
dfRDD = d.partitionBy(n,partitioner)

In [6]:
dRDD = dfRDD.map(lambda r : r[1])

In [7]:
dRDD.glom().map(len).collect()

[366496, 16789, 19587, 45818, 122647]

In [7]:
dat = dRDD.toDF()

In [24]:
dat.write.mode("overwrite").json("/home/shivam/intern/partition")

In [8]:
import json

part0 = "/home/shivam/intern/partition/part-00000-b9624116-91df-4aaa-b88a-04b66e6c4e74-c000.json"

reviews1 = []
with open(part0) as p0:
    for line in p0:
        review = json.loads(line)
        reviews1.append(Review(review['reviewText'],review['overint']))

In [29]:
reviews2[16788].text



"Bought this for my son's 20 in mountain bike and it didn't fit.  No problem I thought, I'll use it on my wife's bike, or my bike.  It didn't fit those either.  Seems to be made for a specific application, not universal at all."

In [9]:
import json

part1 = "/home/shivam/intern/partition/part-00001-b9624116-91df-4aaa-b88a-04b66e6c4e74-c000.json"

reviews2 = []
with open(part1) as p1:
    for line in p1:
        review = json.loads(line)
        reviews2.append(Review(review['reviewText'],review['overint']))

In [10]:
import json

part2 = "/home/shivam/intern/partition/part-00002-b9624116-91df-4aaa-b88a-04b66e6c4e74-c000.json"

reviews3 = []
with open(part2) as p2:
    for line in p2:
        review = json.loads(line)
        reviews3.append(Review(review['reviewText'],review['overint']))

In [11]:
import json

part3 = "/home/shivam/intern/partition/part-00003-b9624116-91df-4aaa-b88a-04b66e6c4e74-c000.json"

reviews4 = []
with open(part3) as p3:
    for line in p3:
        review = json.loads(line)
        reviews4.append(Review(review['reviewText'],review['overint']))

In [12]:
import json

part4 = "/home/shivam/intern/partition/part-00004-b9624116-91df-4aaa-b88a-04b66e6c4e74-c000.json"

reviews5 = []
with open(part4) as p4:
    for line in p4:
        review = json.loads(line)
        reviews5.append(Review(review['reviewText'],review['overint']))

In [27]:
reviews1[316789].text

"I can't stress enough how well these thermos koozies work. I own a can-am Maverick UTV and the cup holders in that thing will boil your drink in 5 minutes. Using these koozies I can set a drink in the cup holder inside this koozie and it will keep it nice and cold for a long time. Well worth the money! The only problem I had was that the bottom plastic piece fell off when I dropped it about 6 ' onto concrete, I just put some epoxy on it and it is as good as new."

In [43]:
import json

part1 = "/home/shivam/intern/parts/part-00001-a2397ecf-c83f-49c7-8ad2-d9e042ffeca9-c000.json"

reviews2 = []
with open(part1) as p1:
    for line in p1:
        review = json.loads(line)
        reviews2.append(Review(review['reviewText'],review['overint']))

In [25]:
import time
start_time = time.time()

from sklearn.model_selection import train_test_split
training,test = train_test_split(reviews5,test_size = 0.25,random_state = 40)

train_x = [x.text for x in training]
train_y = [x.sentiment for x in training]

test_x = [x.text for x in test]
test_y = [x.sentiment for x in test]

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)

In [27]:
from sklearn.ensemble import RandomForestClassifier
clf_dec = RandomForestClassifier()
clf_dec.fit(train_x_vectors,train_y)
x = clf_dec.predict(test_x_vectors)
print(x)
#from sklearn.linear_model import LogisticRegression
#clf_lg = LogisticRegression()
#clf_lg.fit(train_x_vectors,train_y)
#clf_lg.predict(test_x_vectors)

end_time = time.time()

print("--- %s seconds ---" % (end_time - start_time))

['Positive' 'Positive' 'Positive' ... 'Positive' 'Positive' 'Positive']
--- 8.766600608825684 seconds ---


In [42]:
#train_x_vectors.shape
len(train_y)
test_x_vectors.shape

(91624, 81755)